In [ ]:
import sys
import sys
import os
import json
import io
import shutil
import numpy as np
import pandas as pd
import scipy
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neighbors import KernelDensity
from sklearn.metrics import roc_curve
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from stat_util import score_ci
from analysis_utils import *
    
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

# Font color for all figures
FONTCOLOR = 'rgba(0.4,0.4,0.4,1.0)'
GRIDCOLOR = 'rgba(1.0,1.0,1.0,0.3)'
FONTSIZE = 16

SUM_COLS = ['Model', 'Group', 'Accuracy', 'MCC', 'f1', 'AUC', 'TN', 'FP', 'FN', 'TP', 'N']
SCALE_COLS = ['v4_normalized_valence_mean_pt', 'v4_normalized_valence_mean_th',
              'v4_normalized_arousal_mean_pt', 'v4_normalized_arousal_mean_th']
KEEP_COLS = ['treatment', 'responder_wk3', 'pca_ebi0'] + SCALE_COLS # + EBI_ITEMS
OUTMAP = dict(responder_wk3='Week 3')

BASEDIR = './'
TABDIR = f'{BASEDIR}/tables'
FIGDIR = f'{BASEDIR}/figures'
DATDIR = f'{BASEDIR}/data'
os.makedirs(TABDIR, exist_ok=True)
os.makedirs(FIGDIR, exist_ok=True)
print(f'BASEDIR for data, figures, and tables: {BASEDIR}')

SHOW_FIGS = True
USE_LOCAL_DATA_FILE = True

## Load Data

In [ ]:
if USE_LOCAL_DATA_FILE:
    df = pd.read_csv(f'{DATDIR}/final_dataframe.csv').set_index('participant_id')
else:
    assert False, 'ERROR: Must be executed from internal notebook to pull data from DB.'

print(df.shape)
df.head()

## Model Definition

In [ ]:
models = {'EBI': 'OUTCOME ~ 1 + pca_ebi0 ',
          'NLP': 
             ('OUTCOME ~ 1 '
              '+ v4_normalized_arousal_mean_pt + v4_normalized_valence_mean_pt '
              '+ v4_normalized_arousal_mean_th + v4_normalized_valence_mean_th '),
          'EBI and NLP': 
             ('OUTCOME ~ 1 + pca_ebi0 '
              '+ v4_normalized_arousal_mean_pt + v4_normalized_valence_mean_pt '
              '+ v4_normalized_arousal_mean_th + v4_normalized_valence_mean_th '),
          'Full': 
             ('OUTCOME ~ 1 + pca_ebi0 + C(treatment) '
              f'+ v4_normalized_arousal_mean_pt + v4_normalized_valence_mean_pt '
              f'+ v4_normalized_arousal_mean_th + v4_normalized_valence_mean_th '),
         }

## Stats for the full model

In [ ]:
model_fits = {}
for outcome, label in OUTMAP.items():
    print(f'\n*** {outcome} ***')
    model = models['Full'].replace('OUTCOME', outcome)
    tmpdf = df[pd.unique(KEEP_COLS + [outcome])].dropna().copy()
    model_fit, acc_sum, cdf, predicted = fit_logit_statsmodels(tmpdf, model)
    model_fits[outcome] = model_fit
    model_sum = model_fit.summary()
    with open(f'{TABDIR}/{outcome}_logit.tex', 'wt') as fp:
        fp.write(model_sum.as_latex())
    print(model_sum)

In [ ]:
fit_sum_df = pd.DataFrame([{'Model': OUTMAP[l],
                            'Pseudo R-squared': m.prsquared, 
                            'Chi-square': m.llr, 
                            'p-value': m.llr_pvalue, 
                            'df': int(m.df_model),
                            'N': len(m.fittedvalues)} for l, m in model_fits.items()])
fmt = {'Pseudo R-squared': lambda x: f'{x:0.3f}', 'Chi-square': lambda x: f'{x:0.2f}', 
       'p-value': lambda x: f'{x:0.3g}'}
fit_sum_df.style.format(formatter=fmt).to_latex(f'{TABDIR}/fit_sum.tex')
print(fit_sum_df.to_string(index=False, float_format='%0.3f'))

## Logistic Regression using sklearn

In [ ]:
results_all, fit_dfs, sum_dfs = {}, {}, {}
MM = {1: 'Treatment', 2: 'EBI', 3: 'NLP', 4: 'EBI and NLP', 5: 'Full'}
MMR = {v: k for k,v in MM.items()}
for outcome in OUTMAP.keys():
    print(f'\n*** {outcome} ***')
    results_all[outcome] = {k: {} for k in models}
    fit_dfs[outcome] = []
    for label, model_template in models.items():
        model = model_template.replace('OUTCOME', outcome)
        tmpdf = df[pd.unique(KEEP_COLS + [outcome])].dropna().copy()
        with HidePrints():
            sum_str, model_fit, cdf, y_pred_df, endog = fit_logit_sklearn(tmpdf, model, scale_cols=SCALE_COLS)
        results_all[outcome][label] = dict(model_sum=sum_str, model_fit=model_fit, pred_df=y_pred_df, endog=endog)
        cdf['Model'] = label
        fit_dfs[outcome].append(cdf)
    sumdf = pd.concat(fit_dfs[outcome]).reset_index()[SUM_COLS]
    sumdf = sumdf.replace(dict(Model=MMR)).sort_values(['Group', 'Model']).replace(dict(Model=MM))
    sumdf.Group = sumdf.Group.str.replace('COMP360', '')
    sumdf.set_index(['Group', 'Model']).style.format(precision=3).to_latex(f'{TABDIR}/{outcome}_fits.tex')
    print(sumdf.to_string(index=False, float_format='%0.3f'))
    sum_dfs[outcome] = sumdf

In [ ]:
dfs = []
for o, label in OUTMAP.items():
    tmpdf = sum_dfs[o].loc[sumdf.Model == 'Full'].copy()
    tmpdf.Model = label
    dfs.append(tmpdf)
simple_sumdf = pd.concat(dfs).sort_values(['Model']).sort_values(['Group', 'Model'], ascending=False)
txt = simple_sumdf.style.format(precision=3).to_latex()

with open(f'{TABDIR}/all_model_fits.tex', 'wt') as fp:
    fp.write(txt)
print(simple_sumdf.to_string(index=False, float_format='%0.3f'))

## ROC Curves

In [ ]:
rocs = []
ht = ('<br>%{customdata[6]}<br>FP Rate = %{x:.3f}<br>TP Rate = %{y:.3f}'
      '<br>Threshold = %{customdata[0]:.3f}'
      '<br>Accuracy = %{customdata[1]:.1f}%'
      '<br>       True+  True-<br>Pred+  %{customdata[2]:^5}  %{customdata[3]:^5}'
      '<br>Pred-  %{customdata[4]:^5}  %{customdata[5]:^5}<extra></extra>')
for label, res in results_all.items():
    r = res['Full']
    tmpdf = r['pred_df']
    fpr, tpr, thresh = roc_curve(tmpdf[r['endog']], tmpdf.Yes_R)
    cm = []
    for t in thresh:
        pred_bool = tmpdf.Yes_R >= t
        tn, fp, fn, tp = confusion_matrix(tmpdf[r['endog']], pred_bool).ravel()
        acc = round((tp + tn) / (tn + fp + fn + tp) * 100, 2)
        cm.append([t, tp, fp, fn, tn, acc])
    cm = np.array(cm).T
    outdf = pd.DataFrame(data=np.vstack((fpr, tpr, cm)).T, 
                         columns=['False Positive Rate', 'True Positive Rate', 'Threshold', 
                                  'TP', 'FP', 'FN', 'TN', 'Accuracy'])
    outdf['Model'] = OUTMAP[label]
    rocs.append(outdf)

rocsustdf = pd.concat(rocs)

fig = px.line(rocsustdf, x='False Positive Rate', y='True Positive Rate', color='Model', width=450, 
              height=320, hover_data=['Threshold', 'Accuracy', 'TP', 'FP', 'FN', 'TN', 'Model'])
fig.update_traces(line=dict(width=2))
fig.add_shape(type='line', x0=0, y0=0, x1=1, y1=1, xref='x', yref='y', 
              line=dict(color='rgba(0.5,0.5,0.5,0.2)', dash="dot"))
gridcolor = 'rgba(1.0,1.0,1.0,0.3)'
fig.update_yaxes(scaleanchor='x', scaleratio=1, range=[0, 1], linecolor='lightgray', mirror=True, 
                 gridcolor=gridcolor)
fig.update_xaxes(constrain='domain', range=[0, 1], linecolor='lightgray', mirror=True, 
                 gridcolor=gridcolor)
fig.update_layout(font=dict(size=FONTSIZE, color=FONTCOLOR), paper_bgcolor='rgba(0,0,0,0)', 
                  plot_bgcolor='rgba(0,0,0,0)', xaxis_title='False Positive Rate', 
                  yaxis_title='True Positive Rate',
                  legend=dict(yanchor="bottom", y=0, xanchor="right", x=0.8, font_size=FONTSIZE - 2),
                  hoverlabel=dict(bgcolor='rgb(0.95,0.95,0.95)', font_size=12, font_family='Courier'))
fig.update_traces(hovertemplate=ht)
fig.update_traces(patch={'line': {'color': 'rgba(0.3,0.3,0.3,0.7)', 'width': 2, 'dash': 'solid'}}, 
                  selector={'legendgroup': 'Week 3'}) 
fig.update_traces(patch={'line': {'color': 'rgba(0.3,0.3,0.3,0.7)', 'width': 2, 'dash': 'dash'}}, 
                  selector={'legendgroup': 'Sustained'}) 
fig['layout']['margin'] = go.layout.Margin(l=0, r=0, b=0, t=0)

fig.write_image(f'{FIGDIR}/roc_responders.svg')
fig.write_image(f'{FIGDIR}/roc_responders.png')
if SHOW_FIGS:
    fig.show(config=dict(toImageButtonOptions=dict(format='png', filename='roc_responders', scale=2)))

### 95% CI on AUC

In [ ]:
res = []
for label, rall in results_all.items():
    r = rall['Full']
    ytrue = r['pred_df'][r['endog']]
    ypred = r['pred_df']['Yes_R']
    auc, lb, ub, scores = score_ci(ytrue, ypred, score_fun=roc_auc_score, n_bootstraps=5000, 
                                   confidence_level=0.95)
    res.append(dict(Model=OUTMAP[label], AUC=auc, Lower=lb, Upper=ub))
aucdf = pd.DataFrame(res)
aucdf.style.format(precision=3).to_latex(f'{TABDIR}/AUC_bootstrap.tex')
print(aucdf.to_string(index=False, float_format='%0.3f'))

## Plot distributions

In [ ]:
BW = 0.1 # .1
fig = make_subplots(rows=len(results_all), cols=1, shared_yaxes=True, y_title='Density')
colors = ['#636EFA', '#EF553B']
for idx, itm in enumerate(results_all.items()):
    r = itm[1]['Full']
    tmpdf = r['pred_df']
    endog = r['endog']
    sl = True if idx == 0 else False
    for resp_val, resp_label in enumerate(['Non-Responder', 'Responder']):
        vals = tmpdf['Yes_R'].loc[tmpdf[endog] == resp_val].values
        x = np.linspace(-1, 2, 3000)
        kde = KernelDensity(kernel="gaussian", bandwidth=BW).fit(vals.reshape(-1, 1))
        y = np.exp(kde.score_samples(x[:, np.newaxis]))
        x, y = fold_prob(x, y)
        plt = go.Scatter(x=x, y=y, fill='tozeroy', name=resp_label, showlegend=sl, 
                         line=dict(color=colors[resp_val]))
        fig.add_trace(plt, row=idx+1, col=1)

for idx, label in enumerate(results_all):
    fig['layout'][f'xaxis{idx + 1}']['title'] = f'{OUTMAP[label]} Model Predicted Responder Probability'
    
#fig.update_layout(xaxis_title=f'{OUTMAP[label]} Model Predicted Responder Probability',
fig.update_layout(width=700, height=200 * len(results_all), font=dict(size=FONTSIZE, color=FONTCOLOR), 
                  paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                  legend=dict(font_size=FONTSIZE - 2, yanchor="top", y=0.99, xanchor="right", x=0.99))
fig['layout']['annotations'][0]['font']['size'] = FONTSIZE + 4
fig.update_yaxes(showgrid=False, showline=False, zerolinecolor='rgba(0,0,0,0)')
fig.update_xaxes(showgrid=False, showline=False, zerolinecolor='rgba(0,0,0,0)')
fig['layout']['margin'] = go.layout.Margin(l=80, r=0, b=0, t=0)

fig.write_image(f'{FIGDIR}/probdist_all.svg')
fig.write_image(f'{FIGDIR}/probdist_all.png')
if SHOW_FIGS:
    fig.show(config=dict(toImageButtonOptions=dict(format='png', filename='probdist_all', scale=2)))

## Odds ratio for treatment alone

In [ ]:
model = 'responder_wk3 ~ 1 + C(treatment) '
tmpdf = df[KEEP_COLS].dropna().copy()
tmpdf.loc[tmpdf.treatment == 0, 'treatment'] = '1 mg'
tmpdf.loc[tmpdf.treatment == 1, 'treatment'] = '10 mg'
tmpdf.loc[tmpdf.treatment == 2, 'treatment'] = '25 mg'
md = smf.logit(formula=model, data=tmpdf)
res = md.fit()
res_sum = res.summary()
with open(f'{TABDIR}/treatment_logit_3cat.tex', 'wt') as fp:
    fp.write(res_sum.as_latex())
print(res_sum)

params = res.params
conf = np.exp(res.conf_int())
conf['Odds Ratio'] = np.exp(params)
conf.columns = ['5%', '95%', 'Odds Ratio']
conf.style.format(precision=3).to_latex(f'{TABDIR}/treatment_odds_3cat.tex')
print(conf.to_string(float_format='%0.3f'))